In [1]:
cd '/Users/lav/Desktop/darkflow'

/Users/lav/Desktop/darkflow


In [ ]:
!pip install tensorflow==1.13.2

In [5]:
from darkflow.net.build import TFNet
import numpy as np
import cv2
import tensorflow as tf

# Config TF, set True if using GPU
config = tf.ConfigProto(log_device_placement = False)
config.gpu_options.allow_growth = False 

with tf.Session(config=config) as sess:
    options = {
            'model': './cfg/yolo.cfg',
            'load': './bin/yolo.weights',
            'threshold': 0.3,
            #'gpu': 1.0 # uncomment these if using GPU
               }
    tfnet = TFNet(options)    

Parsing ./cfg/yolo.cfg
Loading ./bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.03953719139099121s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
Instructions for updating:
Colocations handled automatically by placer.
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2 

In [6]:
import cv2

In [7]:
capture=cv2.VideoCapture(0)

In [8]:
def Display_Results(results,img):
    for (i,result) in enumerate(results):
        x=result['topleft']['x']
        w=result['bottomright']['x']-result['topleft']['x']
        y=result['topleft']['y']
        h=result['bottomright']['y']-result['topleft']['y']
        cv2.rectangle(img,(x,y),(x+w,y+h),(0.255,0),2)
        label_position = (x+int(w/2),abs(y-10))
        cv2.putText(img,result['label'],label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),2)
    return img

In [ ]:
while True:
    ret , frame = capture.read()
    if ret:
        results=tfnet.return_predict(frame)
        image=Display_Results(results,frame)
        cv2.imshow('Yolo Object Detection',image)
        if cv2.waitKey(1) == 13:
            break
capture.release()
cv2.destroyAllWindows()